In [343]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Date
from sqlalchemy.orm import relationship, backref, declarative_base, sessionmaker
from faker import Faker
import random
import datetime as datetime

Create a db file in SQLite

Create a declarative base class and a faker instance

In [344]:
Base = declarative_base()

fake = Faker(['es_MX'])
engine = create_engine('sqlite:///example2.db', echo=True)

Now create al the classes that defines the tables of our db

In [345]:
class Users(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)

    last_name = Column(String)
    first_name = Column(String)
    email = Column(String)
    address = Column(String)
    city = Column(String)
    state = Column(String)
    zip_code = Column(String)
    user_role = Column(String)

    #Define a relationship to the Performance class
    performance = relationship('Performance', backref=backref('users', lazy=True))

    def __repr__(self):
        return f'<User(name={self.first_name}, email={self.email})>'
    
class Performance(Base):
    __tablename__ = 'performance'

    perf_id = Column(Integer, primary_key=True)

    date = Column(String)
    ensemble_id = Column(Integer, ForeignKey('ensemble.ensemble_id'))
    tab_id = Column(Integer, ForeignKey('scores.tab_id'))
    user_id = Column(Integer, ForeignKey('users.id'))



In [346]:
class Scores(Base):

    __tablename__ = 'scores'

    tab_id = Column(Integer, primary_key=True)

    improvisation = Column(Integer)
    style = Column(Integer)
    presentation = Column(Integer)
    musicality = Column(Integer)
    ensemble = Column(Integer)
    intonation = Column(Integer)
    technique = Column(Integer)

        #Define a relationship to the Performance class
    performance = relationship('Performance', backref=backref('scores', lazy=True))





In [347]:
class School(Base):
    
    __tablename__ = 'school'

    school_id = Column(Integer, primary_key=True)

    school_name = Column(String)
    phone_number = Column(String)
    school_address = Column(String)
    zip_code = Column(String)
    city = Column(String)
    state = Column(String)

    #Define a relationship to the Ensemble class
    ensemble = relationship('Ensemble', backref=backref('school', lazy=True))

    def __repr__(self):
        return f'<User(school_name={self.school_name}, phone={self.phone_number})>'


In [348]:

class Director(Base):
    __tablename__ = 'director'

    director_id = Column(Integer, primary_key=True)

    last_name = Column(String)
    first_name = Column(String)
    phone = Column(String)
    email = Column(String)

    #Define a relationship to the Ensemble class
    ensemble = relationship('Ensemble', backref=backref('director', lazy=True))

    def __repr__(self):
        return f'<director_name={self.first_name} {self.last_name}, phone={self.phone}, email={self.email}>'
    

In [349]:
class Ensemble(Base):
    
    __tablename__ = 'ensemble'

    ensemble_id = Column(Integer, primary_key=True)

    class_ = Column(String)
    number_in_ensemble = Column(Integer)
    school_id = Column(Integer, ForeignKey('school.school_id'))
    director_id = Column(Integer, ForeignKey('director.director_id'))

        #Define a relationship to the Performance class
    performance = relationship('Performance', backref=backref('ensemble', lazy=True))



In [350]:
Base.metadata.create_all(bind=engine)

Session = sessionmaker(bind=engine)
session = Session()

2023-02-28 16:28:52,480 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-28 16:28:52,480 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-02-28 16:28:52,481 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 16:28:52,482 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2023-02-28 16:28:52,482 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 16:28:52,483 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("performance")
2023-02-28 16:28:52,483 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 16:28:52,484 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("performance")
2023-02-28 16:28:52,484 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 16:28:52,485 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("scores")
2023-02-28 16:28:52,486 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-28 16:28:52,487 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("scores")
2023-02-28 16:28:52,487 INFO sqlalchemy.engine.Engine [raw sql] 

In [351]:
classes = ['music', 'acting', 'harmony']

with engine.connect() as conn:
    # create 10 users
    users = [Users(
        last_name=fake.last_name(),
        first_name = fake.first_name(),
        email=fake.email(),
        address=fake.city(),
        state=fake.state(),
        zip_code=fake.postcode(),
        user_role=fake.company_suffix()
    ) for _ in range(2)]
    conn.execute(Users.__table__.insert(), [u.__dict__ for u in users])

    school = [School(
        school_name = f'Colegio {random.randint(1,100)}',
        phone_number = fake.phone_number(),
        school_address = fake.address(),
        zip_code = fake.postcode(),
        city = fake.city(),
        state = fake.state()
    ) for _ in range(1)]
    conn.execute(School.__table__.insert(), [sc.__dict__ for sc in school])

    director = [Director(
        last_name = fake.last_name(),
        first_name = fake.first_name(),
        phone = fake.phone_number(),
        email = fake.email()
    )for _ in range(1)]
    conn.execute(Director.__table__.insert(), [d.__dict__ for d in director])

    scores = [Scores(
        improvisation = random.randint(5, 11),
        style = random.randint(5, 11),
        presentation = random.randint(5, 11),
        musicality = random.randint(5, 11),
        ensemble = random.randint(5, 11),
        intonation = random.randint(5, 11),
        technique = random.randint(5, 11)
    ) for _ in range(1)]
    conn.execute(Scores.__table__.insert(), [s.__dict__ for s in scores])

    ensemble = [Ensemble(
        class_ = random.choice(classes),
        number_in_ensemble = random.randint(0,10),
        school_id = school[i % 10],
        director_id = director[i % 10]
    ) for i in range(1)]
    conn.execute(Ensemble.__table__.insert(), [e.__dict__ for e in ensemble])

            # 10 entries in performance
    performance = [Performance(
        date=fake.date(),
        ensemble_id=ensemble[i % 10],
        tab_id=scores[i % 10],
        user_id=users[i % 10]
    ) for i in range(1)]
    conn.execute(Performance.__table__.insert(), [p.__dict__ for p in performance])

2023-02-28 16:28:52,606 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-28 16:28:52,606 INFO sqlalchemy.engine.Engine INSERT INTO users (last_name, first_name, email, address, state, zip_code, user_role) VALUES (?, ?, ?, ?, ?, ?, ?)
2023-02-28 16:28:52,607 INFO sqlalchemy.engine.Engine [generated in 0.00144s] [('de la Fuente', 'Nayeli', 'patricio34@example.org', 'San Felipe los altos', 'Veracruz de Ignacio de la Llave', '35411-1100', 'S.C.'), ('Linares', 'Ana', 'fabiola66@example.org', 'San Felipe los altos', 'Tabasco', '86307-0721', 'S.C.')]
2023-02-28 16:28:52,609 INFO sqlalchemy.engine.Engine INSERT INTO school (school_name, phone_number, school_address, zip_code, city, state) VALUES (?, ?, ?, ?, ?, ?)
2023-02-28 16:28:52,610 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ('Colegio 10', '(807)900-7675x37332', 'Calzada Pakistán 404 Interior 591\nSan Augusto de la Montaña, CAMP 17075', '45326', 'Nueva Myanmar', 'San Luis Potosí')
2023-02-28 16:28:52,611 INFO sqlalchemy.e

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 2 - probably unsupported type.
[SQL: INSERT INTO ensemble (class_, number_in_ensemble, school_id, director_id) VALUES (?, ?, ?, ?)]
[parameters: ('acting', 5, <User(school_name=Colegio 10, phone=(807)900-7675x37332)>, <director_name=Susana Montenegro, phone=559.541.6837x96427, email=pfajardo@example.com>)]
(Background on this error at: https://sqlalche.me/e/20/rvf5)

In [355]:
print(school[0].school_id, director[0].director_id, users[1].id)

None None None
